# Appendix A) SQLite Warehousing of Stocks, Options, & Fundamental Data

This script is a supplement in which the Dask is used to distribute API calls to store 11gb or so of stocks, options, and fundamental data. 

Data is gathered from:
- The EOD Historical Data API: https://eodhistoricaldata.com/
- Unofficial Python SDK for EODHD: https://github.com/LautaroParada/eod-data
- OECD.Stats
<br>

## Standard Imports and General Settings

In [ ]:
from eod import EodHistoricalData #Python SDK for EODHD API - extract stock data
import timeit #To time processes

# Data Processing
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn', override copywithsetting
import numpy as np
import csv
import time
import datetime # required for creating date-time arrays or unix time codes for EODHD intra-day
from flatten_json import flatten
import json
from textwrap import wrap #string processing
import sqlite3 as sql
import pickle

# Data Storage
import csv
import h5py # For writing, reading, managing results as hdf5 files
from pandas import HDFStore
import h5glance # For visualizing HDF5 file structures and data types
from h5glance import H5Glance
import tables #pytables

# Plotting
import matplotlib as mpl #Matplot lib
import matplotlib.pyplot as plt #pyplot
import plotly.express as px #plotly express
import plotly.io as pio
import plotly.graph_objects as go #for candlestick charts
import kaleido #sub-library required for saving plotly
import seaborn as sns #seabprm
sns.set() 

# API requests
import requests

# Multi-Processing
from dask import delayed
from dask.distributed import Client
from dask.diagnostics import ProgressBar #view progress

In [ ]:
root = r'C:\Users\kevin\Documents\WCD_Bootcamp\ML\ML_Capstone\Stocks_Options\Scripts'
root_raw = r'C:\Users\kevin\Documents\WCD_Bootcamp\ML\ML_Capstone\Stocks_Options\Scripts\ScrapedData'
root_results = r'C:\Users\kevin\Documents\WCD_Bootcamp\ML\ML_Capstone\Stocks_Options\Scripts\Results'

In [ ]:
# API key
EOD_key ='YOUR-KEY-HERE'

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

## Ticker List

Get tickers for different exchanges or general index tickers

In [ ]:
api_key = EOD_key
client = EodHistoricalData(api_key)

In [ ]:
response = client.get_exchanges()

NASDAQ

In [ ]:
NASDAQ = client.get_exchange_symbols(exchange='NASDAQ')

In [ ]:
NASDAQ_tickers = pd.json_normalize(NASDAQ)

In [ ]:
NASDAQ_tickers[(NASDAQ_tickers['Type']=='ETF')&(NASDAQ_tickers['Code'].str.contains('SPX',na=False))]

In [ ]:
NASDAQ_tickers[(NASDAQ_tickers['Type']=='ETF')&(NASDAQ_tickers['Code'].str.contains('SPY',na=False))]

In [ ]:
NASDAQ_tickers[(NASDAQ_tickers['Type']=='ETF')&(NASDAQ_tickers['Code'].str.contains('QQQ',na=False))]

In [ ]:
NASDAQ_tickers[(NASDAQ_tickers['Type']=='ETF')&(NASDAQ_tickers['Isin'].str.contains('US46090E1038',na=False))]

NYSE

In [ ]:
NYSE = client.get_exchange_symbols(exchange='NYSE')

In [ ]:
NYSE_tickers = pd.json_normalize(NYSE)

In [ ]:
NYSE_tickers[(NYSE_tickers['Type']=='ETF')&(NYSE_tickers['Code'].str.contains('SPX',na=False))]

In [ ]:
NYSE_tickers[(NYSE_tickers['Type']=='ETF')&(NYSE_tickers['Code'].str.contains('SPY',na=False))]

In [ ]:
NYSE_tickers[(NYSE_tickers['Type']=='ETF')&(NYSE_tickers['Code'].str.contains('SPY',na=False))]

Government Bonds

In [ ]:
GBonds = client.get_exchange_symbols(exchange='GBOND')

In [ ]:
GBonds_tickers = pd.json_normalize(GBonds)

In [ ]:
GBonds_tickers['Country'].unique()

In [ ]:
tickers_bonds = GBonds_tickers['Code']

TSX

In [ ]:
TSX = client.get_exchange_symbols(exchange='TO')

In [ ]:
TSX_tickers = pd.json_normalize(TSX)

In [ ]:
TSX_tickers.to_csv('TSX_tickers.csv')

In [ ]:
tickers_tsx = TSX_tickers['Code']
tickers_tsx = [t+'.TO' for t in tickers_tsx]

INDX

In [ ]:
INDX = client.get_exchange_symbols(exchange='INDX')

In [ ]:
INDX_tickers = pd.json_normalize(INDX)

US Indices

In [ ]:
us_indices = INDX_tickers[INDX_tickers['Country']=='USA']

In [ ]:
us_indices.to_csv('us_indices.csv')

In [ ]:
us_indices_tickers = us_indices['Code']
us_indices_tickers = [t+'.INDX' for t in us_indices_tickers]

## Listings Data

Query Specific Listings

In [ ]:
resp = client.get_fundamental_equity('QQQ.US') # ETF

In [ ]:
response = client.get_search_instrument(query_string='SPY.US')

In [ ]:
response = client.get_search_instrument(query_string='SPX')

In [ ]:
response = client.get_search_instrument(query_string='.GBOND')

## ETF/Bond/Stock Fundamental Data

Get Fundamental data for main ETFs - get constituents for gathering data on individual stocks

In [ ]:
SPY_const = ['AAPL.US', 'MSFT.US', 'AMZN.US', 'GOOGL.US', 'BRK-B.US', 'GOOG.US', 'UNH.US', 'TSLA.US', 'XOM.US', 'JNJ.US', 'NVDA.US', 'JPM.US', 'PG.US', 'V.US', 'HD.US', 'CVX.US', 'MA.US', 'LLY.US', 'ABBV.US', 'PFE.US', 'MRK.US', 'BAC.US', 'META.US', 'PEP.US', 'KO.US', 'COST.US', 'WMT.US', 'TMO.US', 'AVGO.US', 'MCD.US', 'CSCO.US', 'ACN.US', 'ABT.US', 'DIS.US', 'WFC.US', 'DHR.US', 'LIN.XETRA', 'BMY.US', 'NEE.US', 'VZ.US', 'TXN.US', 'COP.US', 'CMCSA.US', 'ADBE.US', 'CRM.US', 'PM.US', 'AMGN.US', 'HON.US', 'RTX.US', 'QCOM.US', 'T.US', 'LOW.US', 'IBM.US', 'NKE.US', 'UPS.US', 'UNP.US', 'CVS.US', 'GS.US', 'NFLX.US', 'ORCL.US', 'SCHW.US', 'CAT.US', 'ELV.US', 'DE.US', 'SPGI.US', 'MS.US', 'AMD.US', 'INTC.US', 'LMT.US', 'SBUX.US', 'BLK.US', 'INTU.US', 'ADP.US', 'GILD.US', 'PLD.US', 'MDT.US', 'AMT.US', 'BA.US', 'CI.US', 'GE.US', 'TJX.US', 'ISRG.US', 'C.US', 'PYPL.US', 'AXP.US', 'AMAT.US', 'MDLZ.US', 'TMUS.US', 'CB.US', 'ADI.US', 'MMC.US', 'EOG.US', 'NOW.US', 'MO.US', 'VRTX.US', 'REGN.US', 'BKNG.US', 'SYK.US', 'PGR.US', 'NOC.US', 'DUK.US', 'TGT.US', 'SLB.US', 'MMM.US', 'SO.US', 'ZTS.US', 'PNC.US', 'CSX.US', 'APD.US', 'HUM.US', 'BDX.US', 'ETN.US', 'FISV.US', 'AON.US', 'ITW.US', 'MU.US', 'CL.US', 'BSX.US', 'CME.US', 'MPC.US', 'LRCX.US', 'TFC.US', 'WM.US', 'EQIX.US', 'CCI.US', 'PXD.US', 'USB.US', 'ICE.US', 'MRNA.US', 'NSC.US', 'SHW.US', 'GM.US', 'DG.US', 'GD.US', 'EMR.US', 'F.US', 'MCK.US', 'ADM.US', 'ORLY.US', 'VLO.US', 'KLAC.US', 'PSX.US', 'FCX.US', 'OXY.US', 'ATVI.US', 'MET.US', 'HCA.US', 'D.US', 'SRE.US', 'EL.US', 'SNPS.US', 'AZO.US', 'GIS.US', 'AEP.US', 'CNC.US', 'CTVA.US', 'APH.US', 'MCO.US', 'EW.US', 'PSA.US', 'A.US', 'AIG.US', 'ROP.US', 'JCI.US', 'CDNS.US', 'KMB.US', 'NXPI.US', 'TRV.US', 'DVN.US', 'MSI.US', 'MAR.US', 'BIIB.US', 'SYY.US', 'DXCM.US', 'LHX.US', 'ADSK.US', 'ENPH.US', 'CHTR.US', 'FDX.US', 'CMG.US', 'AJG.US', 'AFL.US', 'MCHP.US', 'TT.US', 'MSCI.US', 'WMB.US', 'IQV.US', 'STZ.US', 'TEL.US', 'ROST.US', 'FIS.US', 'O.US', 'PRU.US', 'HES.US', 'MNST.US', 'EXC.US', 'SPG.US', 'CTAS.US', 'PH.US', 'PAYX.US', 'NUE.US', 'COF.US', 'HLT.US', 'XEL.US', 'CARR.US', 'DOW.US', 'KMI.US', 'ECL.US', 'ALL.US', 'NEM.US', 'PCAR.US', 'YUM.US', 'AMP.US', 'DD.US', 'CMI.US', 'FTNT.US', 'HSY.US', 'IDXX.US', 'ED.US', 'EA.US', 'ILMN.US', 'BK.US', 'ANET.US', 'RMD.US', 'OTIS.US', 'TDG.US', 'AME.US', 'MTD.US', 'HAL.US', 'KDP.US', 'WELL.US', 'KR.US', 'VICI.US', 'DLR.US', 'SBAC.US', 'DLTR.US', 'ON.US', 'KEYS.US', 'CSGP.US', 'ALB.US', 'CTSH.US', 'CEG.US', 'PPG.US', 'KHC.US', 'WEC.US', 'ROK.US', 'MTB.US', 'FAST.US', 'DFS.US', 'PEG.US', 'WBA.US', 'OKE.US', 'BKR.US', 'VRSK.US', 'ES.US', 'APTV.US', 'TROW.US', 'GPN.US', 'RSG.US', 'BAX.US', 'STT.US', 'CPRT.US', 'AWK.US', 'HPQ.US', 'GWW.US', 'IT.US', 'ODFL.US', 'WTW.US', 'FANG.US', 'DHI.US', 'GPC.US', 'ABC.US', 'GLW.US', 'IFF.US', 'CDW.US', 'WBD.US', 'URI.US', 'TSCO.US', 'CBRE.US', 'FITB.US', 'EBAY.US', 'ZBH.US', 'HIG.US', 'PCG.US', 'EFX.US', 'EIX.US', 'FTV.US', 'VMC.US', 'WY.US', 'AVB.US', 'ULTA.US', 'RJF.US', 'ETR.US', 'LUV.US', 'ARE.US', 'NDAQ.US', 'MLM.US', 'FRC.US', 'AEE.US', 'DAL.US', 'LYB.US', 'DTE.US', 'LEN.US', 'CTRA.US', 'HBAN.US', 'LH.US', 'CF.US', 'FE.US', 'RF.US', 'CAH.US', 'PPL.US', 'EQR.US', 'IR.US', 'PFG.US', 'ACGL.US', 'ANSS.US', 'PWR.US', 'MKC.US', 'EXR.US', 'CFG.US', 'MRO.US', 'HPE.US', 'DOV.US', 'XYL.US', 'WAT.US', 'EPAM.US', 'TSN.US', 'NTRS.US', 'TDY.US', 'CNP.US', 'CHD.US', 'MOH.US', 'AES.US', 'HOLX.US', 'VRSN.US', 'EXPD.US', 'MAA.US', 'INVH.US', 'WAB.US', 'K.US', 'CLX.US', 'DRI.US', 'AMCR.US', 'VTR.US', 'STE.US', 'MOS.US', 'SYF.US', 'IEX.US', 'CAG.US', 'BALL.US', 'CMS.US', 'KEY.US', 'CINF.US', 'DGX.US', 'PKI.US', 'FDS.US', 'BR.US', 'ABMD.US', 'SEDG.US', 'WST.US', 'NTAP.US', 'TRGP.US', 'ATO.US', 'MPWR.US', 'BBY.US', 'FMC.US', 'SJM.US', 'OMC.US', 'TTWO.US', 'J.US', 'AVY.US', 'EQT.US', 'PAYC.US', 'IRM.US', 'WRB.US', 'COO.US', 'JBHT.US', 'EXPE.US', 'APA.US', 'ETSY.US', 'TXT.US', 'SWKS.US', 'LDOS.US', 'AKAM.US', 'TRMB.US', 'INCY.US', 'TER.US', 'FLT.US', 'HRL.US', 'LKQ.US', 'LVS.US', 'ALGN.US', 'UAL.US', 'ZBRA.US', 'GRMN.US', 'HWM.US', 'LNT.US', 'JKHY.US', 'DPZ.US', 'KIM.US', 'NVR.US', 'BRO.US', 'ESS.US', 'PEAK.US', 'IPG.US', 'TYL.US', 'EVRG.US', 'VTRS.US', 'NLOK.US', 'CBOE.US', 'MTCH.US', 'IP.US', 'RE.US', 'BF-B.US', 'PTC.US', 'RCL.US', 'TECH.US', 'POOL.US', 'HST.US', 'SNA.US', 'SIVB.US', 'NDSN.US', 'CPT.US', 'PKG.US', 'LW.US', 'SWK.US', 'CHRW.US', 'UDR.US', 'CRL.US', 'MGM.US', 'WDC.US', 'MAS.US', 'HSIC.US', 'L.US', 'NI.US', 'VFC.US', 'KMX.US', 'CPB.US', 'EMN.US', 'STX.US', 'GL.US', 'CZR.US', 'TFX.US', 'TAP.US', 'CE.US', 'JNPR.US', 'PHM.US', 'CDAY.US', 'ALLE.US', 'BXP.US', 'BWA.US', 'PARA.US', 'REG.US', 'NRG.US', 'QRVO.US', 'LYV.US', 'MKTX.US', 'FOXA.US', 'WRK.US', 'CCL.US', 'CMA.US', 'TPR.US', 'AAL.US', 'FFIV.US', 'BBWI.US', 'HII.US', 'AAP.US', 'UHS.US', 'ROL.US', 'IVZ.US', 'SBNY.US', 'FBHS.US', 'BIO.US', 'PNW.US', 'RHI.US', 'WHR.US', 'AOS.US', 'HAS.US', 'WYNN.US', 'ZION.US', 'SEE.US', 'FRT.US', 'PNR.US', 'CTLT.US', 'BEN.US', 'NWSA.US', 'AIZ.US', 'DXC.US', 'NCLH.US', 'XRAY.US', 'GNRC.US', 'OGN.US', 'LNC.US', 'ALK.US', 'LUMN.US', 'MHK.US', 'NWL.US', 'RL.US', 'FOX.US', 'DVA.US', 'DISH.US', 'VNO.US', 'NWS.US', 'VNT.US']
QQQ_const = ['AAPL.US', 'MSFT.US', 'AMZN.US', 'GOOG.US', 'GOOGL.US', 'NVDA.US', 'TSLA.US', 'PEP.US', 'COST.US', 'META.US', 'AVGO.US', 'CSCO.US', 'TMUS.US', 'TXN.US', 'CMCSA.US', 'ADBE.US', 'AMGN.US', 'HON.US', 'QCOM.US', 'NFLX.US', 'AMD.US', 'INTC.US', 'SBUX.US', 'INTU.US', 'ADP.US', 'GILD.US', 'ISRG.US', 'PYPL.US', 'MDLZ.US', 'AMAT.US', 'ADI.US', 'VRTX.US', 'REGN.US', 'BKNG.US', 'MRNA.US', 'CSX.US', 'FISV.US', 'MU.US', 'CHTR.US', 'LRCX.US', 'ATVI.US', 'MNST.US', 'ORLY.US', 'KDP.US', 'KLAC.US', 'MAR.US', 'PANW.US', 'ASML.US', 'SNPS.US', 'AEP.US', 'MELI.US', 'KHC.US', 'CTAS.US', 'CDNS.US', 'NXPI.US', 'PAYX.US', 'BIIB.US', 'LULU.US', 'DXCM.US', 'ADSK.US', 'ENPH.US', 'MCHP.US', 'FTNT.US', 'ROST.US', 'EXC.US', 'AZN.US', 'ABNB.US', 'XEL.US', 'MRVL.US', 'PCAR.US', 'EA.US', 'WBA.US', 'ILMN.US', 'IDXX.US', 'DLTR.US', 'ODFL.US', 'CEG.US', 'CTSH.US', 'CPRT.US', 'PDD.US', 'CRWD.US', 'FAST.US', 'WDAY.US', 'VRSK.US', 'SIRI.US', 'JD.US', 'EBAY.US', 'SGEN.US', 'DDOG.US', 'ANSS.US', 'VRSN.US', 'ZS.US', 'ZM.US', 'TEAM.US', 'BIDU.US', 'LCID.US', 'ALGN.US', 'SWKS.US', 'MTCH.US', 'SPLK.US', 'NTES.US', 'DOCU.US']
SPXL_const = ['AAPL.US', 'MSFT.US', 'AMZN.US', 'GOOGL.US', 'BRK-B.US', 'GOOG.US', 'UNH.US', 'TSLA.US', 'XOM.US', 'JNJ.US', 'NVDA.US', 'JPM.US', 'PG.US', 'V.US', 'HD.US', 'CVX.US', 'MA.US', 'LLY.US', 'ABBV.US', 'PFE.US', 'MRK.US', 'BAC.US', 'PEP.US', 'META.US', 'KO.US', 'COST.US', 'WMT.US', 'TMO.US', 'AVGO.US', 'MCD.US', 'CSCO.US', 'ACN.US', 'ABT.US', 'DIS.US', 'WFC.US', 'DHR.US', 'LIN.XETRA', 'BMY.US', 'NEE.US', 'VZ.US', 'TXN.US', 'COP.US', 'CMCSA.US', 'ADBE.US', 'CRM.US', 'PM.US', 'AMGN.US', 'HON.US', 'RTX.US', 'QCOM.US', 'T.US', 'LOW.US', 'IBM.US', 'GS.US', 'NKE.US', 'UPS.US', 'UNP.US', 'CVS.US', 'NFLX.US', 'ORCL.US', 'SCHW.US', 'CAT.US', 'ELV.US', 'DE.US', 'SPGI.US', 'MS.US', 'AMD.US', 'INTC.US', 'LMT.US', 'SBUX.US', 'BLK.US', 'INTU.US', 'ADP.US', 'GILD.US', 'PLD.US', 'MDT.US', 'AMT.US', 'BA.US', 'CI.US', 'GE.US', 'TJX.US', 'ISRG.US', 'C.US', 'PYPL.US', 'AXP.US', 'AMAT.US', 'MDLZ.US', 'TMUS.US', 'CB.US', 'ADI.US', 'MMC.US', 'EOG.US', 'NOW.US', 'BKNG.US', 'MO.US', 'VRTX.US', 'REGN.US', 'SYK.US', 'PGR.US', 'DUK.US', 'TGT.US', 'NOC.US', 'SLB.US', 'MMM.US', 'SO.US', 'ZTS.US', 'APD.US', 'CSX.US', 'PNC.US', 'HUM.US', 'BDX.US', 'ETN.US', 'FISV.US', 'AON.US', 'ITW.US', 'MU.US', 'CL.US', 'BSX.US', 'CME.US', 'MPC.US', 'TFC.US', 'LRCX.US', 'WM.US', 'CCI.US', 'PXD.US', 'EQIX.US', 'USB.US', 'ICE.US', 'NSC.US', 'MRNA.US', 'SHW.US', 'GM.US', 'DG.US', 'GD.US', 'EMR.US', 'F.US', 'ORLY.US', 'MCK.US', 'ADM.US', 'VLO.US', 'KLAC.US', 'PSX.US', 'FCX.US', 'OXY.US', 'ATVI.US', 'HCA.US', 'AZO.US', 'SRE.US', 'MET.US', 'D.US', 'EL.US', 'SNPS.US', 'GIS.US', 'AEP.US', 'CNC.US', 'MCO.US', 'PSA.US', 'APH.US', 'CTVA.US', 'EW.US', 'A.US', 'AIG.US', 'JCI.US', 'ROP.US', 'CDNS.US', 'KMB.US', 'NXPI.US', 'CMG.US', 'TRV.US', 'MAR.US', 'DVN.US', 'MSI.US', 'SYY.US', 'DXCM.US', 'LHX.US', 'BIIB.US', 'CHTR.US', 'ADSK.US', 'ENPH.US', 'FDX.US', 'AJG.US', 'MCHP.US', 'AFL.US', 'STZ.US', 'TT.US', 'WMB.US', 'TEL.US', 'IQV.US', 'HES.US', 'MSCI.US', 'ROST.US', 'PRU.US', 'FIS.US', 'O.US', 'PH.US', 'MNST.US', 'EXC.US', 'SPG.US', 'PAYX.US', 'COF.US', 'CTAS.US', 'NUE.US', 'XEL.US', 'HLT.US', 'CARR.US', 'DOW.US', 'KMI.US', 'ECL.US', 'ALL.US', 'PCAR.US', 'NEM.US', 'AMP.US', 'YUM.US', 'DD.US', 'CMI.US', 'ED.US', 'FTNT.US', 'IDXX.US', 'ILMN.US', 'EA.US', 'HSY.US', 'RMD.US', 'BK.US', 'ANET.US', 'TDG.US', 'OTIS.US', 'HAL.US', 'AME.US', 'KDP.US', 'ALB.US', 'SBAC.US', 'DLTR.US', 'WELL.US', 'VICI.US', 'KEYS.US', 'KR.US', 'DLR.US', 'ON.US', 'CEG.US', 'CSGP.US', 'CTSH.US', 'PPG.US', 'WEC.US', 'KHC.US', 'ROK.US', 'MTD.US', 'FAST.US', 'MTB.US', 'WBA.US', 'DFS.US', 'PEG.US', 'OKE.US', 'BKR.US', 'APTV.US', 'ES.US', 'VRSK.US', 'TROW.US', 'BAX.US', 'STT.US', 'GPN.US', 'RSG.US', 'CPRT.US', 'AWK.US', 'HPQ.US', 'WTW.US', 'IT.US', 'GWW.US', 'ODFL.US', 'DHI.US', 'GPC.US', 'FANG.US', 'GLW.US', 'IFF.US', 'ABC.US', 'WBD.US', 'CDW.US', 'ZBH.US', 'FITB.US', 'TSCO.US', 'CBRE.US', 'URI.US', 'EBAY.US', 'PCG.US', 'HIG.US', 'EIX.US', 'EFX.US', 'VMC.US', 'FTV.US', 'AVB.US', 'WY.US', 'MLM.US', 'ULTA.US', 'LUV.US', 'ARE.US', 'RJF.US', 'ETR.US', 'AEE.US', 'FRC.US', 'NDAQ.US', 'DAL.US', 'DTE.US', 'LEN.US', 'LYB.US', 'HBAN.US', 'CTRA.US', 'CF.US', 'ACGL.US', 'ANSS.US', 'FE.US', 'IR.US', 'CAH.US', 'PPL.US', 'LH.US', 'PFG.US', 'EQR.US', 'RF.US', 'MKC.US', 'PWR.US', 'CFG.US', 'EXR.US', 'MRO.US', 'XYL.US', 'HPE.US', 'DOV.US', 'WAT.US', 'EPAM.US', 'TDY.US', 'NTRS.US', 'CNP.US', 'TSN.US', 'CHD.US', 'AES.US', 'MOH.US', 'HOLX.US', 'VRSN.US', 'K.US', 'MAA.US', 'INVH.US', 'WAB.US', 'EXPD.US', 'CLX.US', 'VTR.US', 'AMCR.US', 'DRI.US', 'FDS.US', 'STE.US', 'MOS.US', 'IEX.US', 'SYF.US', 'CMS.US', 'CAG.US', 'KEY.US', 'BALL.US', 'CINF.US', 'BR.US', 'DGX.US', 'PKI.US', 'WST.US', 'ATO.US', 'ABMD.US', 'TRGP.US', 'SEDG.US', 'NTAP.US', 'BBY.US', 'FMC.US', 'OMC.US', 'J.US', 'SJM.US', 'EQT.US', 'TTWO.US', 'COO.US', 'MPWR.US', 'IRM.US', 'AVY.US', 'PAYC.US', 'EXPE.US', 'WRB.US', 'JBHT.US', 'TXT.US', 'ETSY.US', 'APA.US', 'SWKS.US', 'LDOS.US', 'AKAM.US', 'UAL.US', 'TRMB.US', 'INCY.US', 'FLT.US', 'ALGN.US', 'LKQ.US', 'TER.US', 'HRL.US', 'LNT.US', 'ZBRA.US', 'LVS.US', 'JKHY.US', 'KIM.US', 'HWM.US', 'GRMN.US', 'BRO.US', 'ESS.US', 'DPZ.US', 'TECH.US', 'PEAK.US', 'NLOK.US', 'TYL.US', 'RE.US', 'EVRG.US', 'VTRS.US', 'IPG.US', 'MTCH.US', 'POOL.US', 'BF-B.US', 'IP.US', 'CBOE.US', 'HST.US', 'PTC.US', 'SNA.US', 'SIVB.US', 'RCL.US', 'NDSN.US', 'PKG.US', 'CPT.US', 'SWK.US', 'UDR.US', 'CRL.US', 'LW.US', 'CHRW.US', 'WDC.US', 'MGM.US', 'L.US', 'MAS.US', 'VFC.US', 'NI.US', 'HSIC.US', 'KMX.US', 'CPB.US', 'CZR.US', 'TFX.US', 'STX.US', 'NVR.US', 'EMN.US', 'GL.US', 'TAP.US', 'CE.US', 'JNPR.US', 'PHM.US', 'CDAY.US', 'BXP.US', 'REG.US', 'PARA.US', 'ALLE.US', 'BWA.US', 'NRG.US', 'QRVO.US', 'LYV.US', 'MKTX.US', 'CCL.US', 'FOXA.US', 'WRK.US', 'CMA.US', 'AAL.US', 'HII.US', 'TPR.US', 'ROL.US', 'AAP.US', 'BBWI.US', 'FFIV.US', 'SBNY.US', 'IVZ.US', 'FBHS.US', 'UHS.US', 'PNW.US', 'RHI.US', 'BIO.US', 'HAS.US', 'FRT.US', 'AOS.US', 'WHR.US', 'ZION.US', 'WYNN.US', 'BEN.US', 'PNR.US', 'SEE.US', 'CTLT.US', 'NWSA.US', 'NCLH.US', 'AIZ.US', 'DXC.US', 'XRAY.US', 'GNRC.US', 'OGN.US', 'ALK.US', 'LNC.US', 'LUMN.US', 'MHK.US', 'NWL.US', 'RL.US', 'FOX.US', 'VNO.US', 'DVA.US', 'DISH.US', 'NWS.US']

In [ ]:
# Combine constituent List
constit_list = SPY_const + QQQ_const + SPXL_const
# Function to remove duplicates from list
def remove_list_duplicates(x):
    '''
    Function to remove duplicates from list
    Input: list
    Output: list without duplicates
    '''
    return list(dict.fromkeys(x))
# Remove duplicates from contituent list
constit_list = remove_list_duplicates(constit_list)

Grab all ETF fundamentals and store as pkl for later use

In [ ]:
@delayed #dask delayed decorator
# Function to get fundamentals from EODHistoricalData API
def EODH_get_fundamentals(tickers,api_key,root_save):
    '''
    Calls EODHD using subscription api_key to get historical data on stock fundamentals for specified stock tickers
    Reference: https://eodhistoricaldata.com/financial-apis/stock-options-data/
    See for API inputs: https://github.com/LautaroParada/eod-data#stock-market-prices-splits-and-dividends-data-api-arrow_up
    Input:
        tickers = list of strings for 'ticker' symbols; each in format of '{SYMBOL_NAME}.{EXCHANGE_ID}'; {EXCHANGE ID} may be omitted in some cases
        api_key = API key registered with a subscription to EODHD
        root_save = folder where extracted data will be stored
    Output: 
        errors = list of tickers for which data was not extracted
        success = list of tickers for which data was extracted
        Data saved as pikle files in root_save folder
    '''
    # Initialize the API client
    client = EodHistoricalData(api_key)
    # Initialize lists to store tickers for which data was extracted and not extracted
    errors = []
    success = []
    # Loop through all input tickers
    for t, ticker in enumerate(tickers):
        try:
            # Call the API 
            response = client.get_fundamental_equity(ticker)
            # Save as pkle
            ticker_id = ticker.replace(".","_") #ticker tag without .
            # Save the dfs to pickle for each ticker
            fid = r'\Pickle_Fundamentals\ETF_Fundamentals_' + ticker_id+'.pkl'
            with open(root_save+fid, 'wb') as handle:
                pickle.dump(response, handle, protocol=pickle.HIGHEST_PROTOCOL)
            success.append(ticker)
        except:
            errors.append(ticker)
            continue
    return [success,errors] #return flags for which runs worked and which didn't

In [ ]:
# General arguments
gen_args_fundamentals = {'api_key':EOD_key,'root_save':root_raw,'tickers':'xxx'}

In [ ]:
# Initialize empty list to store results and to pass to dask
results_fundamentals = []
# Loop to initiate dask delayed function
for ticker in ph_tickers: #ETF_list, constit_list
    gen_args_fundamentals.update({'tickers':[ticker]}) #update current ticket in general arguments
    res = EODH_get_fundamentals(**gen_args_fundamentals) #call API function with passed args
    results_fundamentals.append(res) #append results to list

In [ ]:
# Parallel Processing with Dask
c = Client()
results_fundamentals = c.compute(results_fundamentals, sync=True)

## Options Data

In [ ]:
Top100ETFs = pd.read_excel(root_raw+r'\PopularETFs.xls')

In [ ]:
ETF_list = Top100ETFs['Symbol']

In [ ]:
ETF_list = [t+'.US' for t in ETF_list]

In [ ]:
@delayed #dask delayed decorator
def EODH_get_options(tickers,start_date,end_date,api_key,root_save,basedb,contrdb,f_append='replace'):
    '''
    Calls EODHD using subscription api_key to get historical data on options trades for specified stock tickers
    Reference: https://eodhistoricaldata.com/financial-apis/stock-options-data/
    See for API inputs: https://github.com/LautaroParada/eod-data#stock-market-prices-splits-and-dividends-data-api-arrow_up
    Input:
        tickers = list of strings for 'ticker' symbols; each in format of '{SYMBOL_NAME}.{EXCHANGE_ID}'; {EXCHANGE ID} may be omitted in some cases
        start_date = string in date format 'YYYY-MM-DD' for beginning of range to be extracted
        end_date = string in date format 'YYYY-MM-DD' for beginning of range to be extracted
        api_key = API key registered with a subscription to EODHD
        root_save = folder where extracted data will be stored
        basedb, contrdb = options base and contracts SQLite database files respectively
        f_append = flag to 'replace' or 'append' tables in sql
    Output: 
        errors = list of tickers for which data was not extracted
        success = list of tickers for which data was extracted
        Data saved to input SQLite database files & pickle files
    '''
    # Initialize API client
    client = EodHistoricalData(api_key)
    # Initialize lists to store tickers for which data was extracted and not extracted
    errors = []
    success = []
    # Loop through every ticker in input list
    for t, ticker in enumerate(tickers):
        try:
            # Call the API 
            response = client.get_stock_options(ticker, from_=start_date, to=end_date)
            # Normalize to begin parsing the deeply nested dictionary
            df_options_in=pd.json_normalize(response,record_path=['data']) 
            # Extract nested list & dictionary containing individual put and call contracts corresponding to each 
            # ...expiritation date line in the df_options_in
            for i in df_options_in.index: #for each expiration date
                puts_list = [] #initialize empty df list and concat at the end only
                calls_list = [] #repeat for calls@
                for i in df_options_in.index: #for each expiration date
                    # Extract PUT options contracts corresponding to the expiration date
                    # nested list of dictionaries
                    try:  
                        puts_list.append(pd.json_normalize(df_options_in['options.PUT'].iloc[i])) # use json_normalize to convert to df, append to list
                    except:
                        continue
                    try:
                        calls_list.append(pd.json_normalize(df_options_in['options.CALL'].iloc[i])) #repeat for calls
                    except:
                        continue
                try:
                    df_puts = pd.concat(puts_list) #concatenate all the puts dfs of each contract exp. date
                except:
                    continue
                try:
                    df_calls = pd.concat(calls_list) #repeat for calls
                except:
                    df_calls = pd.DataFrame(columns=df_puts.keys())
                    continue
        except Exception:
            print("Error with ticker "+ticker) #flag error and continue
            errors.append(ticker)
            continue
            
        # Clean up the base and contracts tables
        df_options_in = df_options_in.drop(['options.PUT'],axis=1,errors='ignore') #drop the nested dictionary-lists of contracts
        df_options_in = df_options_in.drop(['options.CALL'],axis=1,errors='ignore') #drop the nested dictionary-lists of contracts
        # Add a tag of the origin ticker as a column to the dfs
        ticker_tag = [ticker]*len(df_options_in) #generate a column with the ticker
        df_options_in.insert (0, "Ticker", ticker_tag)
        #calls_islocal = "df_calls" in locals() #check to see if df_calls was created
        #puts_islocal = "df_puts" in locals()
        #if calls_islocal: # using locals() function for checking existence in symbol table
        ticker_tag = [ticker]*len(df_calls)
        df_calls.insert (0, "Ticker", ticker_tag)
        ticker_tag = [ticker]*len(df_puts) # repeat for puts 
        df_puts.insert (0, "Ticker", ticker_tag)
        df_contracts = pd.concat([df_calls,df_puts])
        # Reset the index and delete the extra column
        df_contracts = df_contracts.reset_index()
        df_contracts = df_contracts.drop(['index'],axis=1)
        
        ticker_id = ticker.replace(".","_") #ticker tag without .
        # Save the dfs to pickle for each ticker
        fid = r'\Pickle_Options\OptionsBase_' + ticker_id + '_' + start_date + '_to_' + end_date +'.pkl'
        df_options_in.to_pickle(root_save+fid)
        fid = r'\Pickle_Options\OptionsContracts_' + ticker_id + '_' + start_date + '_to_' + end_date +'.pkl'
        df_contracts.to_pickle(root_save+fid)
        
        #Create SQLite Table - Overwrite if exists
        conn = sql.connect(root_raw+basedb)
        c = conn.cursor()
        df_options_in.to_sql(ticker_id,conn,if_exists=f_append)
        conn.close()
        conn = sql.connect(root_raw+contrdb)
        c = conn.cursor()
        df_contracts.to_sql(ticker_id,conn,if_exists=f_append)
        conn.close()
        print('completed: '+ticker)
        success.append(ticker)
    return [success, errors] #return flags for which runs worked and which didn't

In [ ]:
# Call function to extract data without parallelizing
EODH_get_options(tickers=ETF_list[3:],start_date='2018-01-01',end_date='2019-11-01',api_key=EOD_key,
                                   root_save=root_raw,basedb='\OptionsBase.db',contrdb='\OptionsContracts.db',f_append='replace')

In [ ]:
# Define parameters for function
gen_args = {'start_date':'2018-01-01','end_date':'2019-01-01','api_key':EOD_key,'root_save':root_raw,
            'basedb':'\OptionsBase.db','contrdb':'\OptionsContracts.db','f_append':'append','tickers':'xxx'}

In [ ]:
# Reduced ETF ticket list
ETF_list2 = ['SPY.US','QQQ.US','XLF.US','FXI.US','SH.US','IWM.US','XLE.US','SLV.US','GOVT.US','VWO.US','XLP.US','RSX.US',
            'JNK.US','BKLN.US','SCHF.US','VCIT.US','SPDW.US','BIL.US','SJNK.US','JETS.US','IAU.US','IEF.US','SHY.US',
            'VEU.US','EWG.US','PDBC.US','IGSB.US','IVV.US','PGX.US'] #'USHY.US',

In [ ]:
# Initialize empty list to store results and to pass to dask
results = []
# Loop to initiate dask delayed function
for ticker in constit_list: #ETF_list, ETF_list2
    gen_args.update({'tickers':[ticker]}) #update args with current ticker 
    # Define date ranges - chunk into years
    for start,end in [('2018-01-01','2018-12-31'),('2019-01-01','2019-12-31'),('2020-01-01','2020-12-31'),
                      ('2021-01-01','2021-12-31'),('2022-01-01','2022-11-27')]: #('2018-01-01','2018-12-31'),('2019-01-01','2019-12-31'),('2020-01-01','2020-12-31'),('2021-01-01','2021-12-31'),('2022-01-01','2022-11-27')
        gen_args.update({'start_date':start,'end_date':end})
        res = EODH_get_options(**gen_args)
        results.append(res)

In [ ]:
# Parallel Processing with Dask
c = Client()
results = c.compute(results, sync=True)

## End-of-Day (EOD) Stock/ETF/Forex API

In [ ]:
@delayed #dask delayed decorator
def EODH_get_EOD(tickers, start_date,end_date,increment,api_key,root_save,db,f_append='replace'):
    '''
        Calls EODHD using subscription api_key to get historical data on end-of-day stock price
        Reference: https://eodhistoricaldata.com/financial-apis/api-for-historical-data-and-volumes/
        See for API inputs: https://github.com/LautaroParada/eod-data#stock-market-prices-splits-and-dividends-data-api-arrow_up
        Input:
            tickers = list of strings for 'ticker' symbols; each in format of '{SYMBOL_NAME}.{EXCHANGE_ID}'; {EXCHANGE ID} may be omitted in some cases
            start_date = string in date format 'YYYY-MM-DD' for beginning of range to be extracted
            end_date = string in date format 'YYYY-MM-DD' for beginning of range to be extracted
            increment = increment for ticker data; use ‘d’ for daily, ‘w’ for weekly, ‘m’ for monthly prices.
            api_key = API key registered with a subscription to EODHD
            db = SQLite database file to save data to
            root_save = folder where extracted data will be stored
        Output: 
            errors = list of tickers for which data was not extracted
            success = list of tickers for which data was extracted
             Data saved to input SQLite database files & pickle files
        '''
    # Initialize API client
    client = EodHistoricalData(api_key)
    # Initialize lists to store tickers for which data was extracted and not extracted
    errors = []
    success = []
    # Loop through every ticker in input list
    for t, ticker in enumerate(tickers):
        try:
            # Call the API
            response = client.get_prices_eod(ticker, period=increment, order='a', from_=start_date,to=end_date)
             # Load list containing json into df
            df=pd.json_normalize(response) 
            ticker_tag = [ticker]*len(df) #generate a column with the ticker
            # Add a tag of the origin ticker as a column to the dfs
            ticker_tag = [ticker]*len(df) #generate a column with the ticker
            df.insert (0, "Ticker", ticker_tag)
            df['date'] = df['date'].astype('datetime64[ns]') #ensure datetime64 type to match other dfs
            df=df.astype({"volume":"int64"}) #ensure datetime64 type to match other dfs & for h5 storage

            # Calculate the adjustment factor + adjusted open/high/low - except for currencies
            if not 'FOREX' in ticker:
                df['adjustment_f'] = df['adjusted_close'] / df['close'] 
                df['adjusted_open'] = df['open'] * df['adjustment_f'] 
                df['adjusted_high'] = df['high'] * df['adjustment_f'] 
                df['adjusted_low'] = df['low'] * df['adjustment_f'] 

            ticker_id = ticker.replace(".","_") #ticker tag without .
            # Save the dfs to pickle for each ticker
            fid = r'\Pickle_EOD\EOD_' + ticker_id + '_' + increment+'_'+start_date + '_to_' + end_date +'.pkl'
            df.to_pickle(root_save+fid)

            df = df.fillna(0.0).copy() #fill null values with 0 to avoid write issues
            
            #Create SQLite Table - Overwrite if exists
            conn = sql.connect(root_raw+db)
            c = conn.cursor()
            df.to_sql(ticker_id,conn,if_exists=f_append)
            conn.close()

            success.append(ticker)
            
        except:
            print("Error with ticker "+ticker) #flag error and continue
            errors.append(ticker)
            continue
        
    return [success,errors] #return flags for which runs worked and which didn't

### US Stocks & ETFs

In [ ]:
# Aggregate total list of tickets
total_list = ETF_list + constit_list

In [ ]:
# Define parameters for function
gen_args_EOD = {'start_date':'1972-01-01','end_date':'2022-11-29','increment':'d','api_key':EOD_key,'root_save':root_raw,
            'db':'\DailyTicks.db','f_append':'replace','tickers':'xxx'}

In [ ]:
# Initialize empty list to store results and to pass to dask
results = []
# Loop to initiate dask delayed function
for ticker in total_list: #ETF_list, ETF_list2, constit_list
    gen_args_EOD.update({'tickers':[ticker]}) #update args with current ticker
    res = EODH_get_EOD(**gen_args_EOD) #pass args to function
    results.append(res)

In [ ]:
# Parallel Processing with Dask
c = Client()
results = c.compute(results, sync=True)

### Government Bonds

In [ ]:
# Assemble bond tickers
tickers_bonds = [t+'.GBOND' for t in tickers_bonds]

In [ ]:
# General parameters for function
gen_args_EOD = {'start_date':'1972-01-01','end_date':'2022-11-30','increment':'d','api_key':EOD_key,'root_save':root_raw,
            'db':'\DailyTicks_Bonds.db','f_append':'replace','tickers':'xxx'}

In [ ]:
# Initialize empty list to store results and to pass to dask
results = []
# Loop to initiate dask delayed function
for ticker in tickers_bonds:
    gen_args_EOD.update({'tickers':[ticker]})
    res = EODH_get_EOD(**gen_args_EOD)
    results.append(res)

In [ ]:
# Parallel Processing with Dask
c = Client()
results = c.compute(results, sync=True)

### US Indices

In [ ]:
# General parameters for function
gen_args_EOD = {'start_date':'1972-01-01','end_date':'2022-11-30','increment':'d','api_key':EOD_key,'root_save':root_raw,
            'db':'\DailyTicks_Indices.db','f_append':'replace','tickers':'xxx'}

In [ ]:
# Initialize empty list to store results and to pass to dask
results = []
# Loop to initiate dask delayed function
for ticker in us_indices_tickers:
    gen_args_EOD.update({'tickers':[ticker]})
    res = EODH_get_EOD(**gen_args_EOD)
    results.append(res)

In [ ]:
# Parallel Processing with Dask
c = Client()
results = c.compute(results, sync=True)

### All Canadian Stocks

In [ ]:
# General parameters for function
gen_args_EOD = {'start_date':'1972-01-01','end_date':'2022-11-30','increment':'d','api_key':EOD_key,'root_save':root_raw,
            'db':'\DailyTicks.db','f_append':'replace','tickers':'xxx'}

In [ ]:
# Initialize empty list to store results and to pass to dask
results = []
# Loop to initiate dask delayed function
for ticker in tickers_tsx:
    gen_args_EOD.update({'tickers':[ticker]})
    res = EODH_get_EOD(**gen_args_EOD)
    results.append(res)

In [ ]:
## Parallel Processing with Dask
c = Client()
results = c.compute(results, sync=True)

### PH STocks & ETFs

In [ ]:
# Get Phippine Stock Exchange Tickers
resp = client.get_exchange_symbols(exchange='PSE')

In [ ]:
# convert responses to df
ph_stocks = pd.json_normalize(resp) 

In [ ]:
# Save to csv
ph_stocks.to_csv('PSE_StockList.csv')

In [ ]:
# Load csv
ph_stocks = pd.read_csv('PSE_StockList.csv')

In [ ]:
# Generate ticket code
ph_tickers = ph_stocks['Code']
ph_tickers = [t+'.PSE' for t in ph_tickers]

In [ ]:
results = []
for ticker in ph_tickers: #ETF_list, ETF_list2, constit_list
    gen_args_EOD.update({'tickers':[ticker]})
    res = EODH_get_EOD(**gen_args_EOD)
    results.append(res)

In [ ]:
# Parallel Processing with Dask
c = Client()
results = c.compute(results, sync=True)

### End-of-Day (EOD for Currencies)

Define currencies to examine - create currency pairs

In [ ]:
# Generate currency pair & inverse pair tickets
from itertools import combinations
currencies = ['EUR','USD','CAD','JPY','CHF','INR','NZD','AUD','GBP','BRL','CNY','PHP']
curr_pairs = ["".join(map(str, comb)) for comb in combinations(currencies, 2)] #Create currency pairs 
curr_pairs_alt = [wrap(c,3)[1]+wrap(c,3)[0] for c in curr_pairs] #alternative currency pair order as EOD has missing data
tickers = [c+'.FOREX' for c in curr_pairs]
tickers_alt = [c+'.FOREX' for c in curr_pairs_alt] #alternative order tickers

In [ ]:
# General parameters for function
gen_args_EOD_forex = {'start_date':'1972-01-01','end_date':'2022-11-29','increment':'d','api_key':EOD_key,'root_save':root_raw,
            'db':'\DailyTicks.db','f_append':'replace','tickers':'xxx'}

In [ ]:
# Initialize empty list to store results and to pass to dask
results = []
# Loop to initiate dask delayed function
for ticker in tickers_alt: #ETF_list, ETF_list2, constit_list
    gen_args_EOD_forex.update({'tickers':[ticker]})
    res = EODH_get_EOD(**gen_args_EOD_forex)
    results.append(res)

In [ ]:
# Parallel Processing with Dask
c = Client()
results = c.compute(results, sync=True)

## Intra-day Ticker Data 

https://eodhistoricaldata.com/financial-apis/intraday-historical-data-api/

Function to get intraday stock tick data from the API and a function to input the appropriate unix date-time range

In [ ]:
def datetime_to_unix(dt,direction=0):
    '''Converts string date-time (dt_string) stamp to unix format
    dt = the date time element being converted
    direction = date-time string to unix (0, default) or unix to date-time string (1)
    '''
    if direction == 0: #default convert date-time string to unix format  
        dt_mod = time.mktime(datetime.datetime.strptime(dt, "%Y-%m-%d").timetuple())
    else:
        dt_mod = datetime.datetime.fromtimestamp(dt)
    return dt_mod

### Collect 1min Ticker Data for Stocks

The API only has data in 5-min/1h increments after Oct 2020. To get this fine grained information we need to do 1min increments, for which data is available back to 2004 or so. The API can only process 120 days per call if 1min increments use.

In [ ]:
@delayed #dask delayed decorator
def EODH_get_intraday(tickers,start_date,end_date,freq,increment,api_key,root_save,db,f_append='append'):
    '''
    Calls EODHD using subscription api_key to get historical data on intra-day stock price
    Reference: https://eodhistoricaldata.com/financial-apis/intraday-historical-data-api/
    See for API inputs: https://github.com/LautaroParada/eod-data#stock-market-prices-splits-and-dividends-data-api-arrow_up
    Input:
        tickers = list of strings for 'ticker' symbols; each in format of '{SYMBOL_NAME}.{EXCHANGE_ID}'; {EXCHANGE ID} may be omitted in some cases
        start_date = string in date format 'YYYY-MM-DD' for beginning of range to be extracted
        end_date = string in date format 'YYYY-MM-DD' for beginning of range to be extracted
        freq = string for dateoffset: https://pandas.pydata.org/docs/user_guide/timeseries.html to resample input date range
        increment = increment for ticker data for {1 min, 5 min, 1h} use {'1m','5m','1h'}
            NOTE: The maximum periods between ‘from’ and ‘to’ are 120 days for 1-minute intervals, 
            600 days for 5-minute intervals and 7200 days for 1-hour intervals.
        api_key = API key registered with a subscription to EODHD
        root_save = folder where extracted data will be stored
        db = sqlite3 database file name as string
      Output: 
        errors = list of tickers for which data was not extracted
        success = list of tickers for which data was extracted
        Data saved to input SQLite database files & pickle files
    '''
    # Initialize API client
    client = EodHistoricalData(api_key)
    # Initialize lists to store tickers for which data was extracted and not extracted
    errors = []
    success = []
    # Loop through every ticker in input list
    for t, ticker in enumerate(tickers):
        # Create the date range - separate calls by the necessary increments
        daterange = pd.date_range(start=start_date,end=end_date,freq=freq,inclusive="both")
        daterange = [str(daterange) for daterange in daterange.strftime('%Y-%m-%d')] #convert to string list for input into EOD function
        if end_date not in daterange: #add the final date if last date set is shorter than specified freq
            daterange.append(end_date)
        for m,mo in enumerate(daterange[:-1]): #last element does not need to be run
            try:
                # Call the API - string date-time needs to be converted to unix format
                response = client.get_prices_intraday(ticker, interval=increment,
                                                      from_=datetime_to_unix(daterange[m]), to=datetime_to_unix(daterange[m+1]))
                # Load list containing json into df
                df = pd.json_normalize(response)
                # Convert 'timestamp' column from unix to datetime string by df.apply
                df['timestamp'] = df['timestamp'].apply(datetime.datetime.fromtimestamp)
                # Add a tag of the origin ticker as a column to the dfs
                ticker_tag = [ticker]*len(df) #generate a column with the ticker
                df.insert (0, "Ticker", ticker_tag)
                
                ticker_id = ticker.replace(".","_") #ticker tag without .
                # Save the dfs to pickle for each ticker
                fid = r'\Pickle_IDTicks\IDTicks_' + ticker_id + '_' + increment+'_'+daterange[m] + '_to_' + daterange[m+1] +'.pkl'
                df.to_pickle(root_save+fid)
            
                # Create SQLite Table - Overwrite if exists
                df = df.fillna(0.0).copy() #fill null values with 0 to avoid write issues
                conn = sql.connect(root_raw+db)
                c = conn.cursor()
                df.to_sql(ticker_id,conn,if_exists=f_append)
                conn.close()
                # Flag successes
                success.append((ticker+'_'+daterange[m] + '_to_' + daterange[m+1]))
            except:
                # Flag errors
                errors.append((ticker+'_'+daterange[m] + '_to_' + daterange[m+1]))
                              
    return [success,errors] #return flags for which runs worked and which didn't

In [ ]:
# Define stocks to collect fine-grained data
tickers_stocks_ID = ['NVDA','AMD','TSM','AAPL','AMZN','NFLX','BAC','CVX','OXY','COST','VERU','WYNN','NVAX','UPS','ROKU',
                     'MSFT','SHOP','LVS','FSLR','XOM','CRSR','HAL','WMT','TSLA','TCOM','DIS','AA']
tickers_stocks_ID = [t+'.US' for t in tickers_stocks_ID]

In [ ]:
# Define ETFs to colelct fine-grained data
tickers_ETF_ID = ['SPY','VOO','QQQ','IWM','EEM','SQQQ','TQQQ','XLF','EEM','HYG','GDX','XLE','EFA','TLT','SLV','IEMG','VMO','XBI','XLI',
          'VTEB','USO','AOA','AOR','VIG','VYM','BND','VXUS','GLD','IVV']
tickers_ETF_ID = [t+'.US' for t in tickers_ETF_ID]

In [ ]:
# General parameters for data collection
gen_args_ID = {'start_date':'2004-01-01','end_date':'2022-11-29','freq':'3MS','increment':'1m',
                      'api_key':EOD_key,'root_save':root_raw,'db':'\IntraDayTicks.db','f_append':'append',
                      'tickers':'xxx'}

In [ ]:

# Initialize empty list to store results and to pass to dask
results = []
# Loop to initiate dask delayed function
for ticker in tickers_stocks_ID: #ETF_list, ETF_list2, constit_list, tickers_ETF_ID[:10],tickers_ETF_ID[10:]
    gen_args_ID.update({'tickers':[ticker]}) #update args with current ticker
    res = EODH_get_intraday(**gen_args_ID) #pass args to function
    results.append(res)

In [ ]:
# Parallel Processing with Dask
c = Client()
#with ProgressBar():
results = c.compute(results, sync=True)

In [ ]:
# Examine errors
results_stocks = results
for e in range(len(results_stocks)):
    errors.extend(results_stocks[e][1])

## FINANCIAL NEWS API

In [ ]:
@delayed #dask delayed decorator
def EODH_get_news(tickers,start_date,end_date,freq,api_key,root_save,db,f_append='append'):
    '''
    Calls EODHD using subscription api_key to get historical data on intra-day stock price
    Reference: https://eodhistoricaldata.com/financial-apis/intraday-historical-data-api/
    See for API inputs: https://github.com/LautaroParada/eod-data#stock-market-prices-splits-and-dividends-data-api-arrow_up
    Input:
        tickers = list of strings for 'ticker' symbols; each in format of '{SYMBOL_NAME}.{EXCHANGE_ID}'; {EXCHANGE ID} may be omitted in some cases
        start_date = string in date format 'YYYY-MM-DD' for beginning of range to be extracted
        end_date = string in date format 'YYYY-MM-DD' for beginning of range to be extracted
        freq = string for dateoffset: https://pandas.pydata.org/docs/user_guide/timeseries.html to resample input date range
        api_key = API key registered with a subscription to EODHD
        root_save = folder where extracted data will be stored
        db = sqlite3 database file name as string
      Output: 
        errors = list of tickers for which data was not extracted
        success = list of tickers for which data was extracted
        Data saved to input SQLite database files & pickle files
    '''
    # Initialize API client
    client = EodHistoricalData(api_key)
    # Initialize lists to store tickers for which data was extracted and not extracted
    errors = []
    success = []
    # Loop through every ticker in input list
    for t, ticker in enumerate(tickers):
        # Create the date range - separate calls by the necessary increments
        daterange = pd.date_range(start=start_date,end=end_date,freq=freq,inclusive="both")
        daterange = [str(daterange) for daterange in daterange.strftime('%Y-%m-%d')] #convert to string list for input into EOD function
        if end_date not in daterange: #add the final date if last date set is shorter than specified freq
            daterange.append(end_date)
        for m,mo in enumerate(daterange[:-1]): #last element does not need to be run
            try:
                # Call the API - string date-time needs to be converted to unix format
                response = client.get_financial_news(s=ticker, limit=1000, from_=daterange[m], to=daterange[m+1])
                # Load list containing json into df
                df = pd.json_normalize(response)
                
                ticker_id = ticker.replace(".","_") #ticker tag without .
                # Save the dfs to pickle for each ticker
                fid = r'\Pickle_News\News_' + ticker_id +'_'+daterange[m] + '_to_' + daterange[m+1] +'.pkl'
                df.to_pickle(root_save+fid)
            
                # Create SQLite Table - Overwrite if exists
                df = df.astype({"symbols":"str"}) #lists must be cast as strings to store
                df = df.astype({"tags":"str"}) #lists must be cast as strings to store
                conn = sql.connect(root_raw+db)
                c = conn.cursor()
                df.to_sql(ticker_id,conn,if_exists=f_append)
                conn.close()
                
                # Flag successes
                success.append((ticker+'_'+daterange[m] + '_to_' + daterange[m+1]))
            except:
                # Flag errors
                errors.append((ticker+'_'+daterange[m] + '_to_' + daterange[m+1]))
                              
    return [success,errors] #return flags for which runs worked and which didn't

In [ ]:
# Define parameters for function
gen_args_news = {'start_date':'2020-01-01','end_date':'2022-11-29','freq':'6MS',
                      'api_key':EOD_key,'root_save':root_raw,'db':'\FinancialNews.db','f_append':'append',
                      'tickers':'xxx'}

In [ ]:
# Initialize empty list to store results and to pass to dask
results = []
# Loop to initiate dask delayed function
for ticker in tickers_stocks_ID: #tickers_ETF_ID
    gen_args_news.update({'tickers':[ticker]}) #update args with current ticker
    res = EODH_get_news(**gen_args_news)  #pass args to function
    results.append(res)

In [ ]:
# Parallel Processing with Dask
c = Client()
#with ProgressBar():
results = c.compute(results, sync=True)

## Tweet Sentiment

In [ ]:
@delayed #dask delayed decorator
def EODH_get_tweets(tickers,api_key,root_save):
    '''
    Calls EODHD using subscription api_key to get historical data on tweet sentiment for specified stock tickers
    Reference: https://eodhistoricaldata.com/financial-apis/stock-options-data/
    See for API inputs: https://github.com/LautaroParada/eod-data#stock-market-prices-splits-and-dividends-data-api-arrow_up
    Input:
        tickers = list of strings for 'ticker' symbols; each in format of '{SYMBOL_NAME}.{EXCHANGE_ID}'; {EXCHANGE ID} may be omitted in some cases
        api_key = API key registered with a subscription to EODHD
        root_save = folder where extracted data will be stored
    Output: 
        errors = list of tickers for which data was not extracted
        success = list of tickers for which data was extracted
        Data saved as pikle files in root_save folder
    '''
    # Initialize the API client
    client = EodHistoricalData(api_key)
    # Initialize lists to store tickers for which data was extracted and not extracted
    errors = []
    success = []
    # Loop through all input tickers
    for t, ticker in enumerate(tickers):
        try:
            # Call the API 
            response = client.get_financial_tweets(s=ticker,from_='2021-01-01',to='2022-12-01')
            # Save as pkle
            ticker_id = ticker.replace(".","_") #ticker tag without .
            # Save the dfs to pickle for each ticker
            fid = r'\Pickle_TweetSentiment\Tweets_' + ticker_id+'.pkl'
            with open(root_save+fid, 'wb') as handle:
                pickle.dump(response, handle, protocol=pickle.HIGHEST_PROTOCOL)
            success.append(ticker)
        except:
            errors.append(ticker)
            continue
    return [success,errors] #return flags for which runs worked and which didn't

In [ ]:
# General arguments
gen_args_tweets= {'api_key':EOD_key,'root_save':root_raw,'tickers':'xxx'}

In [ ]:
# Initialize empty list to store results and to pass to dask
results_tweets = []
results_fundamentals = []
# Loop to initiate dask delayed function
for ticker in constit_list: 
    gen_args_tweets.update({'tickers':[ticker]}) #update current ticket in general arguments
    res = EODH_get_tweets(**gen_args_tweets) #call API function with passed args
    results_tweets.append(res) #append results to list

In [ ]:
# Parallel Processing with Dask
c = Client()
results_tweets = c.compute(results_tweets, sync=True)